In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.


In [2]:
X_train = np.random.randint(3, high=16, size=(11000,8))
Y_train = np.sort(X_train, axis=-1)
X_target = np.copy(Y_train)
X_target[:,1:] = X_target[:,:-1]
X_target[:,0] = 1
X_train = np.concatenate([X_train, X_target], axis=-1)
X_train,X_test = X_train[1000:],X_train[:1000]
Y_train,Y_test = Y_train[1000:],Y_train[:1000]
X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

((10000, 16), (10000, 8), (1000, 16), (1000, 8))

In [3]:
data_input_dim = max(np.max(X_train), np.max(X_test)) + 1
data_input_dim

16

In [4]:
data_output_dim = max(np.max(Y_train), np.max(Y_test)) + 1
data_output_dim

16

In [5]:
class DNCCell(keras.layers.Layer):
    
    def __init__(self,
                 output_size,
                 word_size,
                 memory_size,
                 register_size=2,
                 write_head_count=2,
                 read_head_count=2,
                 enable_temporal=True,
                 enable_controller_output_resize=True,
                 enable_final_bias=True,
                 bypass_dropout_factor=None,
                 **kwargs):
        self.output_size = output_size
        self.word_size = word_size
        self.memory_size = memory_size
        self.register_size = register_size
        self.write_head_count = write_head_count
        self.read_head_count = read_head_count
        self.enable_temporal = enable_temporal
        self.enable_controller_output_resize = enable_controller_output_resize
        self.enable_final_bias = enable_final_bias
        self.bypass_dropout_factor = bypass_dropout_factor
        self.state_size = (output_size,)
        self.state_size += (register_size * word_size,)
        self.state_size += (register_size * word_size,)
        self.state_size += (memory_size * word_size,)
        self.state_size += (read_head_count * memory_size,)
        self.state_size += (read_head_count * word_size,)
        self.state_size += (write_head_count * memory_size,)
        self.state_size += (memory_size,)
        if enable_temporal:
            self.state_size += (write_head_count * memory_size,)
            self.state_size += (write_head_count * memory_size**2,)
        super().__init__(**kwargs)
    
    def build(self, input_shape):
        read_vec_size = self.word_size * self.read_head_count
        controller_input_size = input_shape[-1]
        controller_input_size += read_vec_size
        controller_input_size += self.register_size*self.word_size
        controller_kernel_size = self.register_size*self.word_size*4
        controller_hidden_size = self.register_size*self.word_size
        interface_vec_size = read_vec_size
        interface_vec_size += self.read_head_count*2
        if self.enable_temporal:
            interface_vec_size += self.read_head_count*3
        interface_vec_size += self.write_head_count*self.word_size*3
        interface_vec_size += self.write_head_count*3
        self.kernel_controller_hidden = self.add_weight(
            shape=(controller_input_size, controller_kernel_size),
            initializer='glorot_normal', name='kernel_controller_hidden')
        self.bias_controller_hidden = self.add_weight(
            shape=(controller_kernel_size,),
            initializer='zeros', name='bias_controller_hidden')
        self.kernel_controller_output = self.add_weight(
            shape=(controller_hidden_size, self.output_size + interface_vec_size),
            initializer='glorot_normal', name='kernel_controller_output')
        self.kernel_read_vec_to_output = self.add_weight(
            shape=(read_vec_size, self.output_size),
            initializer='glorot_normal', name='kernel_read_vec_to_output')
        if self.enable_controller_output_resize:
            self.scalar_controller_output_resize = self.add_weight(shape=(),
                initializer='ones', name='scalar_controller_output_resize')
        if self.enable_final_bias:
            self.bias_final_output = self.add_weight(
                shape=(self.output_size,),
                initializer='zeros', name='bias_final_output')
        super().build(input_shape)
    
    def call(self, inputs, states, training=None):
        
        def oneplus(x):
            return K.softplus(x) + 1.
        
        def similarity(m, k, b):
            dot = K.batch_dot(k, m, axes=2)
            m_len = K.sqrt(K.sum(K.square(m), axis=-1))
            k_len = K.sqrt(K.sum(K.square(k), axis=-1))
            mk_len = K.expand_dims(k_len, axis=-1) @ K.expand_dims(m_len, axis=-2)
            mk_len = K.switch(K.not_equal(mk_len, 0.), mk_len, K.ones_like(mk_len))
            cos_sim = dot / mk_len
            return K.softmax(cos_sim * K.expand_dims(b))
        
        def batch_invert_permutation(permutations):
            # from https://github.com/deepmind/dnc/blob/master/dnc/util.py
            perm = tf.cast(permutations, tf.float32)
            dim = int(perm.get_shape()[-1])
            size = tf.cast(tf.shape(perm)[0], tf.float32)
            delta = tf.cast(tf.shape(perm)[-1], tf.float32)
            rg = tf.range(0, size * delta, delta, dtype=tf.float32)
            rg = tf.expand_dims(rg, 1)
            rg = tf.tile(rg, [1, dim])
            perm = tf.add(perm, rg)
            flat = tf.reshape(perm, [-1])
            perm = tf.invert_permutation(tf.cast(flat, tf.int32))
            perm = tf.reshape(perm, [-1, dim])
            return tf.subtract(perm, tf.cast(rg, tf.int32))

        def batch_gather(values, indices):
            # from https://github.com/deepmind/dnc/blob/master/dnc/util.py
            idx = tf.expand_dims(indices, -1)
            size = tf.shape(indices)[0]
            rg = tf.range(size, dtype=tf.int32)
            rg = tf.expand_dims(rg, -1)
            rg = tf.tile(rg, [1, int(indices.get_shape()[-1])])
            rg = tf.expand_dims(rg, -1)
            gidx = tf.concat([rg, idx], -1)
            return tf.gather_nd(values, gidx)
        
        _, register_s_last, register_h_last, memory_last, \
            read_weights_last, read_vec_last, write_weights_last, \
            usage_last, *temporal_states = states
        memory_last = K.reshape(memory_last,
            (-1,self.memory_size,self.word_size))
        read_weights_last = K.reshape(read_weights_last,
            (-1,self.read_head_count,self.memory_size))
        write_weights_last = K.reshape(write_weights_last,
            (-1,self.write_head_count,self.memory_size))
        if not self.enable_temporal:
            assert not temporal_states
        else:
            preced_last, link_last = temporal_states
            preced_last = K.reshape(preced_last,
                (-1,self.write_head_count,self.memory_size))
            link_last = K.reshape(link_last,
                (-1,self.write_head_count,self.memory_size,self.memory_size))
        
        # feeding controller with current input and last read vecs
        output = K.concatenate([inputs, read_vec_last, register_h_last])
        output = output @ self.kernel_controller_hidden
        output = output + self.bias_controller_hidden
        ctr_input_gate = output[:,:self.register_size*self.word_size]
        ctr_input_gate = K.sigmoid(ctr_input_gate)
        ctr_forget_gate = output[:,
            self.register_size*self.word_size:
            self.register_size*self.word_size*2]
        ctr_forget_gate = K.sigmoid(ctr_forget_gate)
        register_s = output[:,
            self.register_size*self.word_size*2:
            self.register_size*self.word_size*3]
        register_s = ctr_input_gate * K.tanh(register_s)
        register_s = register_s + ctr_forget_gate*register_s_last
        ctr_output_gate = output[:,
            self.register_size*self.word_size*3:
            self.register_size*self.word_size*4]
        ctr_output_gate = K.sigmoid(ctr_output_gate)
        register_h = ctr_output_gate * K.tanh(register_s)
        output = register_h @ self.kernel_controller_output
        
        # break down controller output into semi final output and interface vec
        interface_vec = output[:,self.output_size:]
        output = output[:,:self.output_size]
        if self.bypass_dropout_factor is not None:
            output = K.in_train_phase(K.dropout(
                output,self.bypass_dropout_factor),output,training=training)
        if self.enable_controller_output_resize:
            output = output * self.scalar_controller_output_resize
        
        # break down interface vec
        interface_pos_last = 0
        interface_partition = []
        for interface_part_len in [
            self.read_head_count * self.word_size,
            self.read_head_count,
            self.write_head_count * self.word_size,
            self.write_head_count,
            self.write_head_count * self.word_size,
            self.write_head_count * self.word_size,
            self.read_head_count,
            self.write_head_count,
            self.write_head_count,
            *([self.read_head_count * 3] if self.enable_temporal else [])]:
            interface_pos = interface_pos_last + interface_part_len
            interface_partition.append(
                interface_vec[:,interface_pos_last:interface_pos])
            interface_pos_last = interface_pos
        read_keys, read_stre, write_keys, write_stre, erase_vecs, \
            write_vecs, free_gates, alloc_gates, write_gates, \
            *temporal_interface_partition = interface_partition
        read_keys = K.reshape(read_keys,(-1,self.read_head_count,self.word_size))
        read_stre = oneplus(read_stre)
        write_keys = K.reshape(write_keys,(-1,self.write_head_count,self.word_size))
        write_stre = oneplus(write_stre)
        erase_vecs = K.reshape(erase_vecs,(-1,self.write_head_count,self.word_size))
        erase_vecs = K.sigmoid(erase_vecs)
        write_vecs = K.reshape(write_vecs,(-1,self.write_head_count,self.word_size))
        free_gates = K.expand_dims(free_gates)
        free_gates = K.sigmoid(free_gates)
        alloc_gates = K.expand_dims(alloc_gates)
        alloc_gates = K.sigmoid(alloc_gates)
        write_gates = K.expand_dims(write_gates)
        write_gates = K.sigmoid(write_gates)
        if not self.enable_temporal:
            assert not temporal_interface_partition
        else:
            read_modes, = temporal_interface_partition
            read_modes = K.reshape(read_modes,(-1,self.read_head_count,3))
            read_modes = K.softmax(read_modes,axis=-1)
        
        # compute allocation vector
        retention = K.prod(1.-(free_gates*read_weights_last), axis=-2)
        # https://github.com/deepmind/dnc/blob/master/dnc/addressing.py
        # according to the deepmind implementation,
        # only write weight is not differentiable
        write_weights_last_nograd = K.stop_gradient(write_weights_last)
        # reduce for multi-write-head, not presented in original papaer
        write_weights_last_nograd = 1.-K.prod(1.-write_weights_last_nograd, axis=-2)
        usage = ((usage_last+write_weights_last_nograd) - \
                usage_last*write_weights_last_nograd) * retention
        # loop for multi-write-head support
        mwh_write_gates = write_gates * alloc_gates
        mwh_usage = usage
        mwh_alloc = []
        for i in range(self.write_head_count):
            # quickfix for tf.cumprod grad bug
            # https://github.com/tensorflow/tensorflow/issues/3862
            usage_qfixed = 1e-6 + (1-1e-6) * mwh_usage
            usage_asc,usage_perm = tf.nn.top_k(1.-usage_qfixed,k=self.memory_size)
            usage_asc = 1.-usage_asc
            alloc_asc = (1.-usage_asc) * tf.cumprod(usage_asc,axis=-1,exclusive=True)
            alloc_perm = batch_invert_permutation(usage_perm)
            alloc = batch_gather(alloc_asc, alloc_perm)
            mwh_alloc.append(alloc)
            mwh_usage += (1.-mwh_usage) * mwh_write_gates[:,i,:] * alloc
        alloc = K.stack(mwh_alloc, axis=-2)
        
        # compute write weight
        write_sims = similarity(memory_last,write_keys,write_stre)
        write_weights = write_gates*(alloc_gates*alloc + (1.-alloc_gates)*write_sims)
        
        # compute precedence and temporal links
        if self.enable_temporal:
            write_weights_rep_h =  K.expand_dims(write_weights,axis=-1)
            write_weights_rep_v =  K.expand_dims(write_weights,axis=-2)
            link = (1.-(write_weights_rep_h+write_weights_rep_v))*link_last + \
                write_weights_rep_h * K.expand_dims(preced_last,axis=-2)
            link = link * (1.-tf.eye(self.memory_size))
            read_weights_last_rep = K.repeat_elements(
                K.expand_dims(read_weights_last,axis=-3),self.write_head_count,-3)
            # reduce sum for multi-write-head
            link_forward = K.sum(read_weights_last_rep @ link,axis=-3)
            link_backward = K.sum(K.permute_dimensions(link @ K.permute_dimensions(
                read_weights_last_rep,(0,1,3,2)),(0,1,3,2)),axis=-3)
            preced = (1.-K.sum(
                write_weights,axis=-1,keepdims=True))*preced_last + write_weights
        
        # update memory
        m_reset = K.expand_dims(write_weights,axis=-1) @ \
            K.expand_dims(erase_vecs,axis=-2)
        # reduce prod for multi-write-head
        m_keep = K.prod(1.-m_reset,axis=-3)
        m_new = K.expand_dims(write_weights,axis=-1) @ \
            K.expand_dims(write_vecs,axis=-2)
        # reduce sum for multi-write-head
        m_new = K.sum(m_new, axis=-3)
        memory = memory_last*m_keep + m_new
        
        # compute read weights and read vectors
        read_sims = similarity(memory,read_keys,read_stre)
        if not self.enable_temporal:
            read_weights = read_sims
        else:
            read_weights = read_modes[:,:,0:1] * read_sims + \
                            read_modes[:,:,1:2] * link_forward + \
                            read_modes[:,:,2:3] * link_backward
        read_vecs = read_weights @ memory
        read_vec = K.reshape(read_vecs, (-1,self.read_head_count*self.word_size))
        
        # compute final output from controller output and current read vecs
        output = output + read_vec @ self.kernel_read_vec_to_output
        if self.enable_final_bias:
            output = output + self.bias_final_output
        
        return output, [
            output,
            register_s,
            register_h,
            K.reshape(memory, (-1,self.memory_size*self.word_size)),
            K.reshape(read_weights, (-1,self.read_head_count*self.memory_size)),
            read_vec,
            K.reshape(write_weights, (-1,self.write_head_count*self.memory_size)),
            usage,
            *([
                K.reshape(preced, (-1,self.write_head_count*self.memory_size)),
                K.reshape(link, (-1,self.write_head_count*self.memory_size**2))
            ] if self.enable_temporal else [])]
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

In [6]:
X = X_inputs = keras.layers.Input((None,), dtype='int32')
X = keras.layers.Embedding(data_input_dim, 16, mask_zero=True)(X)
X = keras.layers.RNN(DNCCell(data_output_dim, 8, 16), return_sequences=True)(X)
X = keras.layers.Lambda(lambda x,s: x[:,s:,:],
    arguments={'s':X_train.shape[1]//2})(X)
X = keras.layers.Softmax()(X)
X = [keras.layers.Lambda(lambda x,s: x[:,s,:], name=f'o{i+1}',
    arguments={'s':i}, output_shape=(data_output_dim,))(X)
    for i in range(X_train.shape[1]//2)]
M = keras.Model(X_inputs, X)
M.compile('nadam', 'sparse_categorical_crossentropy', ['acc'])
M.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 16)     256         input_1[0][0]                    
__________________________________________________________________________________________________
rnn_1 (RNN)                     (None, None, 16)     4945        embedding_1[0][0]                
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 16)     0           rnn_1[0][0]                      
__________________________________________________________________________________________________
softmax_1 

In [7]:
M.fit(X_train, np.split(Y_train,Y_train.shape[-1],axis=-1),
    validation_data=(X_test,np.split(Y_test,Y_test.shape[-1],axis=-1)),
    batch_size=128, epochs=100, callbacks=[
    keras.callbacks.ReduceLROnPlateau('loss', patience=3, verbose=1)
])

Train on 10000 samples, validate on 1000 samples
Epoch 1/100
10000/10000 [==============================] - 25s 2ms/step - loss: 18.9151 - o1_loss: 2.4605 - o2_loss: 2.4534 - o3_loss: 2.4658 - o4_loss: 2.4738 - o5_loss: 2.4407 - o6_loss: 2.3633 - o7_loss: 2.2358 - o8_loss: 2.0217 - o1_acc: 0.1201 - o2_acc: 0.1344 - o3_acc: 0.1313 - o4_acc: 0.1236 - o5_acc: 0.1337 - o6_acc: 0.1346 - o7_acc: 0.1765 - o8_acc: 0.3308 - val_loss: 15.3592 - val_o1_loss: 1.9534 - val_o2_loss: 1.9923 - val_o3_loss: 2.0347 - val_o4_loss: 2.0325 - val_o5_loss: 1.9969 - val_o6_loss: 1.9339 - val_o7_loss: 1.8003 - val_o8_loss: 1.6152 - val_o1_acc: 0.2240 - val_o2_acc: 0.2070 - val_o3_acc: 0.2260 - val_o4_acc: 0.2760 - val_o5_acc: 0.2790 - val_o6_acc: 0.2480 - val_o7_acc: 0.2730 - val_o8_acc: 0.3240
Epoch 2/100
10000/10000 [==============================] - 21s 2ms/step - loss: 14.2194 - o1_loss: 1.7552 - o2_loss: 1.8469 - o3_loss: 1.8969 - o4_loss: 1.8885 - o5_loss: 1.8635 - o6_loss: 1.8365 - o7_loss: 1.7228 - o8_

10000/10000 [==============================] - 21s 2ms/step - loss: 8.8535 - o1_loss: 0.8780 - o2_loss: 1.0102 - o3_loss: 1.0541 - o4_loss: 1.1291 - o5_loss: 1.2065 - o6_loss: 1.2800 - o7_loss: 1.2624 - o8_loss: 1.0332 - o1_acc: 0.6336 - o2_acc: 0.5813 - o3_acc: 0.5527 - o4_acc: 0.5122 - o5_acc: 0.4751 - o6_acc: 0.4498 - o7_acc: 0.4371 - o8_acc: 0.5236 - val_loss: 8.8935 - val_o1_loss: 0.9092 - val_o2_loss: 1.0089 - val_o3_loss: 1.0705 - val_o4_loss: 1.1211 - val_o5_loss: 1.2064 - val_o6_loss: 1.3204 - val_o7_loss: 1.2846 - val_o8_loss: 0.9725 - val_o1_acc: 0.5970 - val_o2_acc: 0.5820 - val_o3_acc: 0.5430 - val_o4_acc: 0.4880 - val_o5_acc: 0.4720 - val_o6_acc: 0.4300 - val_o7_acc: 0.4230 - val_o8_acc: 0.5340
Epoch 13/100
10000/10000 [==============================] - 21s 2ms/step - loss: 8.5495 - o1_loss: 0.8458 - o2_loss: 0.9743 - o3_loss: 1.0104 - o4_loss: 1.0808 - o5_loss: 1.1634 - o6_loss: 1.2397 - o7_loss: 1.2236 - o8_loss: 1.0115 - o1_acc: 0.6542 - o2_acc: 0.6000 - o3_acc: 0.5667

10000/10000 [==============================] - 21s 2ms/step - loss: 6.4891 - o1_loss: 0.5261 - o2_loss: 0.6295 - o3_loss: 0.6871 - o4_loss: 0.7700 - o5_loss: 0.8760 - o6_loss: 0.9801 - o7_loss: 1.0605 - o8_loss: 0.9597 - o1_acc: 0.8308 - o2_acc: 0.7610 - o3_acc: 0.7188 - o4_acc: 0.6679 - o5_acc: 0.6256 - o6_acc: 0.5750 - o7_acc: 0.5106 - o8_acc: 0.5371 - val_loss: 12.0825 - val_o1_loss: 0.6226 - val_o2_loss: 0.9200 - val_o3_loss: 1.4153 - val_o4_loss: 1.9105 - val_o5_loss: 2.1333 - val_o6_loss: 1.9660 - val_o7_loss: 1.8669 - val_o8_loss: 1.2479 - val_o1_acc: 0.7610 - val_o2_acc: 0.6050 - val_o3_acc: 0.4370 - val_o4_acc: 0.3440 - val_o5_acc: 0.3540 - val_o6_acc: 0.3290 - val_o7_acc: 0.3280 - val_o8_acc: 0.4930
Epoch 24/100
10000/10000 [==============================] - 21s 2ms/step - loss: 6.4125 - o1_loss: 0.5017 - o2_loss: 0.6114 - o3_loss: 0.6775 - o4_loss: 0.7667 - o5_loss: 0.8636 - o6_loss: 0.9695 - o7_loss: 1.0647 - o8_loss: 0.9576 - o1_acc: 0.8444 - o2_acc: 0.7702 - o3_acc: 0.726

10000/10000 [==============================] - 21s 2ms/step - loss: 5.1250 - o1_loss: 0.2553 - o2_loss: 0.3916 - o3_loss: 0.4771 - o4_loss: 0.5785 - o5_loss: 0.6824 - o6_loss: 0.8302 - o7_loss: 0.9771 - o8_loss: 0.9329 - o1_acc: 0.9337 - o2_acc: 0.8704 - o3_acc: 0.8192 - o4_acc: 0.7689 - o5_acc: 0.7194 - o6_acc: 0.6449 - o7_acc: 0.5658 - o8_acc: 0.5567 - val_loss: 4.7721 - val_o1_loss: 0.2401 - val_o2_loss: 0.3493 - val_o3_loss: 0.4273 - val_o4_loss: 0.5268 - val_o5_loss: 0.6201 - val_o6_loss: 0.7703 - val_o7_loss: 0.8911 - val_o8_loss: 0.9470 - val_o1_acc: 0.9410 - val_o2_acc: 0.8980 - val_o3_acc: 0.8390 - val_o4_acc: 0.7790 - val_o5_acc: 0.7500 - val_o6_acc: 0.6590 - val_o7_acc: 0.6130 - val_o8_acc: 0.5380
Epoch 35/100
10000/10000 [==============================] - 21s 2ms/step - loss: 4.7934 - o1_loss: 0.2318 - o2_loss: 0.3482 - o3_loss: 0.4255 - o4_loss: 0.5097 - o5_loss: 0.6304 - o6_loss: 0.7890 - o7_loss: 0.9490 - o8_loss: 0.9098 - o1_acc: 0.9439 - o2_acc: 0.8891 - o3_acc: 0.8386

10000/10000 [==============================] - 20s 2ms/step - loss: 3.8175 - o1_loss: 0.1505 - o2_loss: 0.2172 - o3_loss: 0.2644 - o4_loss: 0.3609 - o5_loss: 0.4780 - o6_loss: 0.6626 - o7_loss: 0.8409 - o8_loss: 0.8430 - o1_acc: 0.9650 - o2_acc: 0.9385 - o3_acc: 0.9173 - o4_acc: 0.8640 - o5_acc: 0.8040 - o6_acc: 0.7090 - o7_acc: 0.6036 - o8_acc: 0.5952 - val_loss: 3.7027 - val_o1_loss: 0.1587 - val_o2_loss: 0.2495 - val_o3_loss: 0.3014 - val_o4_loss: 0.3769 - val_o5_loss: 0.5026 - val_o6_loss: 0.6128 - val_o7_loss: 0.7345 - val_o8_loss: 0.7663 - val_o1_acc: 0.9540 - val_o2_acc: 0.9270 - val_o3_acc: 0.8840 - val_o4_acc: 0.8450 - val_o5_acc: 0.7830 - val_o6_acc: 0.7350 - val_o7_acc: 0.6620 - val_o8_acc: 0.6240
Epoch 46/100
10000/10000 [==============================] - 20s 2ms/step - loss: 3.6993 - o1_loss: 0.1438 - o2_loss: 0.2072 - o3_loss: 0.2524 - o4_loss: 0.3480 - o5_loss: 0.4668 - o6_loss: 0.6387 - o7_loss: 0.8133 - o8_loss: 0.8292 - o1_acc: 0.9677 - o2_acc: 0.9452 - o3_acc: 0.9217

10000/10000 [==============================] - 21s 2ms/step - loss: 2.8779 - o1_loss: 0.0919 - o2_loss: 0.1402 - o3_loss: 0.1714 - o4_loss: 0.2494 - o5_loss: 0.3580 - o6_loss: 0.5082 - o7_loss: 0.6501 - o8_loss: 0.7087 - o1_acc: 0.9791 - o2_acc: 0.9642 - o3_acc: 0.9530 - o4_acc: 0.9171 - o5_acc: 0.8630 - o6_acc: 0.7917 - o7_acc: 0.7013 - o8_acc: 0.6613 - val_loss: 2.9574 - val_o1_loss: 0.0781 - val_o2_loss: 0.1260 - val_o3_loss: 0.1661 - val_o4_loss: 0.2643 - val_o5_loss: 0.3942 - val_o6_loss: 0.5191 - val_o7_loss: 0.6362 - val_o8_loss: 0.7734 - val_o1_acc: 0.9840 - val_o2_acc: 0.9730 - val_o3_acc: 0.9470 - val_o4_acc: 0.8980 - val_o5_acc: 0.8400 - val_o6_acc: 0.7680 - val_o7_acc: 0.7180 - val_o8_acc: 0.6190
Epoch 57/100
10000/10000 [==============================] - 21s 2ms/step - loss: 2.7525 - o1_loss: 0.0840 - o2_loss: 0.1292 - o3_loss: 0.1560 - o4_loss: 0.2339 - o5_loss: 0.3390 - o6_loss: 0.4885 - o7_loss: 0.6350 - o8_loss: 0.6869 - o1_acc: 0.9829 - o2_acc: 0.9687 - o3_acc: 0.9586

10000/10000 [==============================] - 21s 2ms/step - loss: 2.0491 - o1_loss: 0.0701 - o2_loss: 0.0961 - o3_loss: 0.1064 - o4_loss: 0.1477 - o5_loss: 0.2199 - o6_loss: 0.3293 - o7_loss: 0.4851 - o8_loss: 0.5945 - o1_acc: 0.9839 - o2_acc: 0.9780 - o3_acc: 0.9751 - o4_acc: 0.9609 - o5_acc: 0.9281 - o6_acc: 0.8777 - o7_acc: 0.7868 - o8_acc: 0.7186 - val_loss: 1.9711 - val_o1_loss: 0.0559 - val_o2_loss: 0.0845 - val_o3_loss: 0.1049 - val_o4_loss: 0.1418 - val_o5_loss: 0.2258 - val_o6_loss: 0.3201 - val_o7_loss: 0.4551 - val_o8_loss: 0.5830 - val_o1_acc: 0.9890 - val_o2_acc: 0.9800 - val_o3_acc: 0.9780 - val_o4_acc: 0.9720 - val_o5_acc: 0.9260 - val_o6_acc: 0.8700 - val_o7_acc: 0.8010 - val_o8_acc: 0.7070
Epoch 68/100
10000/10000 [==============================] - 21s 2ms/step - loss: 1.9282 - o1_loss: 0.0691 - o2_loss: 0.0927 - o3_loss: 0.1037 - o4_loss: 0.1414 - o5_loss: 0.2059 - o6_loss: 0.3103 - o7_loss: 0.4539 - o8_loss: 0.5510 - o1_acc: 0.9826 - o2_acc: 0.9787 - o3_acc: 0.9755

10000/10000 [==============================] - 21s 2ms/step - loss: 1.3304 - o1_loss: 0.0484 - o2_loss: 0.0610 - o3_loss: 0.0694 - o4_loss: 0.0894 - o5_loss: 0.1254 - o6_loss: 0.1884 - o7_loss: 0.3140 - o8_loss: 0.4343 - o1_acc: 0.9898 - o2_acc: 0.9892 - o3_acc: 0.9865 - o4_acc: 0.9803 - o5_acc: 0.9636 - o6_acc: 0.9383 - o7_acc: 0.8627 - o8_acc: 0.7936 - val_loss: 1.3594 - val_o1_loss: 0.0369 - val_o2_loss: 0.0523 - val_o3_loss: 0.0740 - val_o4_loss: 0.0927 - val_o5_loss: 0.1454 - val_o6_loss: 0.1933 - val_o7_loss: 0.2979 - val_o8_loss: 0.4671 - val_o1_acc: 0.9940 - val_o2_acc: 0.9920 - val_o3_acc: 0.9850 - val_o4_acc: 0.9870 - val_o5_acc: 0.9580 - val_o6_acc: 0.9360 - val_o7_acc: 0.8820 - val_o8_acc: 0.7790
Epoch 79/100
10000/10000 [==============================] - 21s 2ms/step - loss: 1.4451 - o1_loss: 0.0471 - o2_loss: 0.0605 - o3_loss: 0.0698 - o4_loss: 0.0904 - o5_loss: 0.1269 - o6_loss: 0.2062 - o7_loss: 0.3604 - o8_loss: 0.4838 - o1_acc: 0.9892 - o2_acc: 0.9886 - o3_acc: 0.9850

10000/10000 [==============================] - 21s 2ms/step - loss: 0.8759 - o1_loss: 0.0334 - o2_loss: 0.0412 - o3_loss: 0.0485 - o4_loss: 0.0604 - o5_loss: 0.0813 - o6_loss: 0.1222 - o7_loss: 0.2043 - o8_loss: 0.2846 - o1_acc: 0.9922 - o2_acc: 0.9927 - o3_acc: 0.9897 - o4_acc: 0.9847 - o5_acc: 0.9794 - o6_acc: 0.9610 - o7_acc: 0.9279 - o8_acc: 0.8934 - val_loss: 0.8896 - val_o1_loss: 0.0360 - val_o2_loss: 0.0454 - val_o3_loss: 0.0503 - val_o4_loss: 0.0725 - val_o5_loss: 0.0887 - val_o6_loss: 0.1159 - val_o7_loss: 0.1842 - val_o8_loss: 0.2966 - val_o1_acc: 0.9950 - val_o2_acc: 0.9900 - val_o3_acc: 0.9930 - val_o4_acc: 0.9810 - val_o5_acc: 0.9780 - val_o6_acc: 0.9680 - val_o7_acc: 0.9500 - val_o8_acc: 0.8660
Epoch 90/100
10000/10000 [==============================] - 20s 2ms/step - loss: 2.9850 - o1_loss: 0.3315 - o2_loss: 0.3183 - o3_loss: 0.3201 - o4_loss: 0.3819 - o5_loss: 0.3826 - o6_loss: 0.4033 - o7_loss: 0.4228 - o8_loss: 0.4246 - o1_acc: 0.9394 - o2_acc: 0.9404 - o3_acc: 0.9429

10000/10000 [==============================] - 21s 2ms/step - loss: 0.5937 - o1_loss: 0.0234 - o2_loss: 0.0297 - o3_loss: 0.0360 - o4_loss: 0.0445 - o5_loss: 0.0649 - o6_loss: 0.0910 - o7_loss: 0.1465 - o8_loss: 0.1578 - o1_acc: 0.9960 - o2_acc: 0.9956 - o3_acc: 0.9924 - o4_acc: 0.9897 - o5_acc: 0.9844 - o6_acc: 0.9721 - o7_acc: 0.9513 - o8_acc: 0.9452 - val_loss: 0.5804 - val_o1_loss: 0.0365 - val_o2_loss: 0.0303 - val_o3_loss: 0.0355 - val_o4_loss: 0.0598 - val_o5_loss: 0.0643 - val_o6_loss: 0.0935 - val_o7_loss: 0.1237 - val_o8_loss: 0.1368 - val_o1_acc: 0.9910 - val_o2_acc: 0.9950 - val_o3_acc: 0.9940 - val_o4_acc: 0.9860 - val_o5_acc: 0.9860 - val_o6_acc: 0.9730 - val_o7_acc: 0.9620 - val_o8_acc: 0.9610
